### this script has been taken from the checkFiltering and DivideTrialsIntoReadingAndWriting script, and so all the filtering is based on the same

In [1]:
%matplotlib qt
#%matplotlib inline
import os
import csv
import numpy as np
import re
import datetime
import fnmatch
from pathlib import Path
from itertools import groupby
import copy
import itertools
#import distance
import nltk
#nltk.download('stopwords')
import pandas as pd
import matplotlib.pyplot as plt
import pickle

from nltk.stem import SnowballStemmer
from nltk.stem import snowball

from itertools import *
from operator import *

from openpyxl import load_workbook

In [32]:
# exceptional removal of particular extra sentences not typed by the user 
dict_phraseStim = {
    #'2019-02-05-14-10-39_2ndPart_2' : [1, 2, 3, 4, 5, 6, 9, 10],
    #'2019-01-14-14-58-30' : [0], # ys, session_trial ()
    '2019-01-16-16-36-17_1stPart_2' : [-1], # af_session1
    '2019-01-16-17-00-12_2ndPart_2': [1], # af_session1
    '2019-01-17-15-27-20_1stPart_2' : [4], # Af session2
    '2019-01-17-16-03-27_2ndPart_2' : [0, 1, 2], # Af session2
    '2019-02-06-11-25-41_1' : [7],               # aq_session1    
    '2019-02-08-11-33-53_1stPart_1' : [1],  # aq session3_1_part1
    '2019-02-08-12-11-34_2ndPart_1' : [0, 1, 2, 3],  # aq session3_1_part2
    '2019-01-31-09-37-5_2ndPart_2' : range(1,5), # bh1, session 4 , all sentences except the first one deleted
    '2019-01-31-09-22-49_1stPart_2' : [4],  # bh1_session4_2_part1
    '2019-02-21-16-09-44_1stPart_1' : [1], # bh2_session1
    '2019-02-21-16-22-22_2ndPart_1' : [2, 3, 4],# bh2_session1
    '2019-02-28-17-03-53_1stPart_2' : [2],       # bh2_session3
    '2019-02-28-17-24-2_2ndPart_2' : [0, 2],     # bh2_session3
    '2019-02-14-13-28-20_1stPart_2' : [2], # cw_session3_2_part1
    '2019-02-14-13-57-41_2ndPart_2' : [0, 2, 3], # cw_session3_2_part2
    '2019-02-21-15-01-4_1stPart_1' : [0],        # le_session3
    '2019-02-21-15-25-56_2ndPart_1' : [1],        # le_session3
    '2019-02-18-10-28-35_2' : [0],               # ls2_session4 # picture not described
    '2019-02-05-14-00-27_1stPart_2' : [3],        # mh_session1
    '2019-02-05-14-10-39_2ndPart_2' : [0, 1, 3],   # mh_session1
    '2019-02-08-10-51-3_1stPart_1' : [4],        # mn_session1
    '2019-02-08-11-05-7_2ndPart_1' : [0, 2, 3, 4], # mn_session1
    '2019-02-19-10-34-7_1stPart_1' : [3],          # mn_session3
    '2019-02-19-10-56-43_2ndPart_1' : [1, 2, 3, 4], # mn_session3
    '2019-01-16-15-18-0_1' : [4],            # no_session1
    '2019-02-19-17-10-45_1' : [3],                  # ph_session5
    '2019-01-29-13-25-4_1' : [3],        # ph_session2
    '2019-03-07-16-44-5_2' : [1],                   # rh_session1
    '2019-03-14-13-56-56_2' : [2],                  # rh_session3
    '2019-01-14-15-07-21_1' : [4], # ys_session1
    '2019-01-16-15-18-50_1stPart_1' : [3, 4], # ys_session2
    '2019-01-16-15-42-51_2ndPart_1' : [2], # ys_session2
    '2019-01-30-11-22-25_1' : [3, 5, 7],          # ys_session4
    '2019-01-30-11-22-25_1' : [4, 6, 7] # ys, session 4
}

# exceptional removal of sentences/words typed by the user, but then deleted everything to have a blank scratchpad

dict_phraseUser = {
    "2019-02-06-15-44-15_1" : [2, 3, 6], 
    "2019-02-06-16-19-9_2" : [1, 3, 6, 7],
    "2019-02-12-11-21-21_2" : [0],
    "2019-02-14-14-28-49_1" : [0, 2, 3], # ac_session3_1
    "2019-02-14-14-45-49_2" : [0, 5, 6], # ac_session3_2
    '2019-01-29-14-19-26_1' : [0, 3, 4], # bh1_session2_1
    '2019-01-29-14-40-36_2' : [0, 1, 2], # bh1_session2_2
    '2019-01-30-14-29-29_2' : [4],       # bh1_session3_2
    '2019-01-31-09-12-2_1' : [3],         # bh1_session4_1
    '2019-01-31-09-22-49_1stPart_2' : [4], # bh1_session4_2_part1
    '2019-03-05-09-15-11_1' : [1],         # bh2_session5_1
    '2019-03-05-09-15-11_2' : [1],        # bh2_session5_2
    '2019-02-21-15-55-56_2' : [2],       # ch_session5_2
    '2019-01-30-15-19-36_2' : [1],       # jm_session2_1
    '2019-01-30-15-04-30_1' : [0],         # jm_session2_2
    '2019-01-16-15-18-50_1stPart_1' : [1],  # ys_session2
    '2019-01-16-15-42-51_2ndPart_1' : [0], # ys_session2
    '2019-01-30-11-22-25_1' : [2, 4],       # ys_session4
    '2019-01-30-11-57-3_2' : [0] ,          # ys_session4
    '2019-01-31-13-13-2_1' : [4],           # ys_session5
    '2019-01-30-10-20-32_1' : [0, 1, 2, 3, 4, 5], # no_session4
    '2019-01-30-10-46-38_2' : [0],          # 
    '2019-02-28-17-03-53_1stPart_2' : [2],   # bh2_session3
    '2019-03-12-09-30-5_1' : [0],            # kj_session3
    '2019-02-13-15-20-38_1' : [0, 1, 2, 3, 6], # ls1_session3
    '2019-02-18-10-25-52_1' : [1],              # ls2_session4
    '2019-02-18-10-46-26_2' : [0],            # ls2_session4
    '2019-01-29-13-25-4_1' : [0, 1, 7],        # ph_session2
    '2019-01-29-13-43-50_2' : [0],              # ph_session2
    '2019-03-07-16-17-30_1' : [0],              # rh_session1
    '2019-03-07-16-44-5_2' : [0, 1],         # rh_session1
    '2019-03-14-13-56-56_2' : [0, 1, 3]         # rh_session3
}

# key selection can have extra selections of NextPhrase at the end
dict_keySelectionOfNextPhrase = {
    "2019-02-11-11-18-30_1" : [12, 13], # ac_session1
    "2019-01-16-17-00-12_2ndPart_2" : [12], # af_session1
    "2019-01-17-15-27-20_1stPart_2" : [12], # af_session2
    "2019-02-06-16-19-9_2" : [12], # af_session3
    "2019-02-12-11-07-43_1" : [12], # af_session4
    "2019-02-27-15-08-32_1" : [12], # af_session5
    "2019-01-28-14-30-44_1" : [12], # bh1_session1
    "2019-02-21-16-22-22_2ndPart_1" : [12], # bh2_session1
    "2019-02-18-14-02-56_2" : [12], # le_session1
    "2019-02-19-10-03-14_1" : [12], # le_session2
    "2019-02-08-11-05-7_2ndPart_1" : [12], # mn_session1
    "2019-02-08-11-12-51_2" : [12, 13], # mn_session1
    "2019-02-15-11-38-22_1" : [12, 13], # mn_session2
    "2019-02-15-11-54-25_2" : [12], # mn_session2
    "2019-01-16-15-18-0_1" : [12], # no_session1
    "2019-01-28-13-31-51_1" : [12], # ph_session1
    "2019-01-28-13-49-14_2" : [12], # ph_session1
    "2019-01-14-15-07-21_1" : [12], # ys_session1
    "2019-01-17-15-05-1_1" : [12], # ys_session3
    "2019-01-30-11-22-25_1" : [12], # ys_session4
    "2019-01-31-13-32-2_2" : [12], # ys_session5
}


# key selection when participants skips some sentences
dict_keySelectionNotCompleted = {
    "2019-01-16-16-36-17_1stPart_2" : [0, 1, 3, 5, 7], # af_session1 ---- last sentence is not finished
    "2019-01-16-17-00-12_2ndPart_2" : [0, 1, 3, 4, 5, 7, 9, 11], # af_session1
    "2019-01-17-15-27-20_1stPart_2" : [0, 1, 3, 5, 7, 9, 11], # af_session2 
    "2019-01-17-16-03-27_2ndPart_2" : [0, 1, 2, 3, 4, 5, 6, 7, 9, 11], # af_session2
    "2019-02-08-11-33-53_1stPart_1" : [0, 1, 3, 4, 5, 7, 9, 11], # aq_session3
    "2019-02-08-12-11-34_2ndPart_1" : [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 11], # aq_session3
    "2019-01-28-14-30-44_1" : [0, 1, 3, 5], # bh1_session1
    "2019-01-31-09-22-49_1stPart_2": [0, 1, 3, 5, 7, 9, 10, 11], # bh1_session4
    "2019-01-31-09-37-5_2ndPart_2" : [0, 1, 3, 4, 5, 6, 7, 8, 9, 10, 11], # bh1_session4
    "2019-02-21-16-09-44_1stPart_1" : [0, 1, 3, 4, 5, 7, 9, 11], # bh2_session1
    "2019-02-21-16-22-22_2ndPart_1" : [0, 1, 3, 5, 6, 7, 8, 9, 10, 11], # bh2_session1
    "2019-02-28-17-03-53_1stPart_2" : [0, 1, 3, 5, 6, 7, 9, 11], # bh2_session3
    "2019-02-28-17-24-2_2ndPart_2" : [0, 1, 2, 3, 5], # bh2_session3     ----
    "2019-02-14-13-28-20_1stPart_2" : [0, 1, 3, 5, 6, 7, 9, 11], # cw_session3
    "2019-02-14-13-57-41_2ndPart_2" : [0, 1, 2, 3, 5, 6, 7, 8, 9, 11], # cw_session3
    "2019-02-21-15-01-4_1stPart_1" : [0, 1, 2, 3, 5, 7, 9, 11], # le_session3
    "2019-02-21-15-25-56_2ndPart_1" : [0, 1, 3], # le_session3       ----
    "2019-02-05-14-00-27_1stPart_2" : [0, 1, 3, 5, 7, 8], # mh_session1
    "2019-02-05-14-10-39_2ndPart_2" : [0, 1, 2, 3, 4, 5, 7, 8, 9, 11], # mh_session1
    "2019-02-08-10-51-3_1stPart_1" : [0, 1, 3, 5, 7, 9, 10, 11], # mn_session1
    "2019-02-08-11-05-7_2ndPart_1" : [0, 1, 2, 3, 5, 6, 7, 8, 9, 10, 11], # mn_session1
    "2019-02-19-10-34-7_1stPart_1" : [0, 1, 3, 5, 7, 8, 9, 11], # mn_session3
    "2019-02-19-10-56-43_2ndPart_1" : [0, 1, 3, 4, 5, 6, 7, 8, 9, 10, 11], # mn_session3
    "2019-01-29-13-25-4_1" : [0, 1, 3, 5, 7], # ph_session2  -- sessions where there are less score questions 
    # and more sentences typed
    "2019-01-16-15-18-50_1stPart_1" : [0, 1, 3, 5, 7, 8, 9, 10], # ys_session2
    "2019-01-17-15-05-1_1" : [0, 1, 3, 5],  # ys_session3  -- sessions where there are less score questions 
    # and more sentences typed
    "2019-02-06-11-25-41_1" : [0, 1, 3, 5, 11], # aq_session1 -- sessions where there are less score questions 
    # and more sentences typed
    "2019-01-16-15-42-51_2ndPart_1" : [0, 1, 2, 5], # ys_session2 -- different for reading and writing, this one is for
    # writing
    '2019-01-30-11-22-25_1' : [0, 1, 3, 5, 7, 9, 11]   # ys_session4 -- sessions where there are less score questions 
    # and more sentences typed
   
}

# dictionary for phrase removal just like in the dict_phraseStim, but since not all participants require that, some that 
# do, are added to this new dictionary here
dict_keySelection_phraseStim = {
    '2019-01-17-15-27-20_1stPart_2' : [4], # Af session2
    '2019-01-16-15-18-0_1' : [4],        # no_session1
    '2019-02-19-17-10-45_1' : [3],                  # ph_session5
    '2019-03-07-16-44-5_2' : [1],        # rh_session1
    '2019-03-14-13-56-56_2' : [2],              # rh_session3
    '2019-01-14-15-07-21_1' : [4]         # ys_session1
}


# in the beginning experiments, not everyone started with 800 initial dwell time

dict_dwellTimeOrig_not800 = {
    "2019-01-16-15-51-13_2" : 600, # no_session1
    "2019-01-16-15-18-0_1" : 600, # no_session1
    "2019-01-16-15-43-8_1" : 100, # af_session1
    "2019-01-16-16-36-17_1stPart_2" : 100, # af_session1
    "2019-01-16-17-00-12_2ndPart_2" : 100, # af_session1
    "2019-01-17-15-03-40_1" : 100, # af_session2
    "2019-01-17-15-27-20_1stPart_2" : 0, # af_session2
    "2019-01-17-16-03-27_2ndPart_2" : 100, # af_session2
    "2019-01-14-15-07-21_1" : 500, # ys_session1
    "2019-01-14-15-25-55_2" : 300, # ys_session1
    "2019-01-16-15-18-50_1stpart_1" : 200, # ys_session2
    "2019-01-16-15-42-51_2ndPart_1" : 100, # ys_session2
    "2019-01-16-15-59-55_2" : 100, # ys_session2
    "2019-01-17-15-05-1_1" : 100, # ys_session3
    "2019-01-17-15-31-12_2" : 100 # ys_session3
}


# list of all things that should be present when computing effective time
list_keysToBeCounted = ['Comma', 'BackOne', 'BackMany', 'SpaceBar']

# some sessions do not have gaze data
dict_noGazeData = {
    '2019-01-16-17-00-12_2ndPart_2' : 'no gaze data', # af_session2
    '2019-01-17-15-31-12_2' : 'no gaze data', #ys_session2
    '2019-01-30-11-57-3_2' : 'no gaze data' # ys_session4
}



In [22]:
dict_keySelection_ReadingTrials = {
    "2019-01-16-15-42-51_2ndPart_1" : [0, 1, 3, 5], # ys_session2 
}

dict_keySelection_WritingTrials = {
    "2019-01-16-15-42-51_2ndPart_1" : [0, 1, 2, 5], # ys_session2   
}

# normally, reading part of trial ends when people look at the keyboardwithphrases. For some trials, this is not done,
# as the reading is done, and the trial is accidentally skipped, and written in the next trial. Here, the trial number 
# given will have the reading time ending as sleep, and not keyboard with phrases. 
dict_keyboardNotChange_ReadingTrials = {
    "2019-01-16-15-42-51_2ndPart_1" : 0, # ys_session2 
}

dict_keySelection_firstSleepNotCounted = {
    "2019-01-28-14-50-41_2" : (0, 2), # bh1_session1 -- 3rd sleep activation to be counted
    "2019-02-19-10-56-43_2ndPart_1" : 2  # mn_session3 -- 3rd sleep activation is to be counted
}

In [23]:
TimeDwellOrig = 800
TimeFixation = 300

In [24]:
def FixUserKeys(UserKeys_Old):
    # Fix the situation where comma has divided decimals into separate columns
    
    Column_beforeDecimal = [item[2] for item in UserKeys_Old]
    Column_afterDecimal = [item[3] if len(item)>3 else '00' for item in UserKeys_Old]
    
    UserKeys_ProgressPercent = [float(Column_beforeDecimal[i]+'.'+ Column_afterDecimal[i]) for i in 
                                range(0, len(Column_beforeDecimal))]
    UserKeys_Times = [item[0] for item in UserKeys_Old]
    UserKeys_Keys = [item[1] for item in UserKeys_Old]
    
    UserKeys_New = [[UserKeys_Times[ind], UserKeys_Keys[ind], UserKeys_ProgressPercent[ind]] for ind in 
                    range(0, len(UserKeys_ProgressPercent))]
    
    #UserKeys_New = np.concatenate((UserKeys_Times, UserKeys_Keys, UserKeys_ProgressPercent), axis = 0)
    
    
    return UserKeys_New
        

In [66]:
def ComputeDwellTime(userKeys, full_path):
    # modify userKeys to include a column of time instead of progress pct, which is dependent on the then dwell time
    
    TimeDwellOrig = 800
    
    # session name
    session_folder_name = full_path.split('\\')[-1]
    
    if session_folder_name in dict_dwellTimeOrig_not800:
        TimeDwellOrig = dict_dwellTimeOrig_not800[session_folder_name]
    
    #print(TimeDwellOrig)
    
    time1, t1, t2 = userKeys[0][0].partition('+')
    timePrevious = datetime.datetime.strptime(re.sub('[:.T]','-',time1[:-1]), "%Y-%m-%d-%H-%M-%S-%f")
    
    dwellTimeList, timeList = list(), list()
    dwellTimeList.append(TimeDwellOrig)
    
    timeDwell = TimeDwellOrig
    nKey = -1
    
    #print('start', userKeys[0][0])
    
    for key in userKeys:
        nKey = nKey + 1
        #print(key[1])
        if key[1] == 'IncreaseDwellTime':
            if float(key[2]) == 1:
                timeDwell = timeDwell + 100
                
                time1, t1, t2 = key[0].partition('+')
                timeNow = datetime.datetime.strptime(re.sub('[:.T]','-',time1[:-1]), "%Y-%m-%d-%H-%M-%S-%f")
                timeChange = (timeNow - timePrevious).total_seconds()
                #print(timePrevious, timeNow)
                
                timePrevious = timeNow
                
                dwellTimeList.append(timeDwell)
                timeList.append(timeChange)
                
                #print('inc', key[0], timeDwell)
                
        elif key[1] == 'DecreaseDwellTime':
            if float(key[2]) == 1:
                
                if userKeys[nKey-1][1] == 'DecreaseDwellTime' and userKeys[nKey-1][2] == 1 and timeDwell == 0:
                    continue
                    
                timeDwell = timeDwell - 100
                
                time1, t1, t2 = key[0].partition('+')
                timeNow = datetime.datetime.strptime(re.sub('[:.T]','-',time1[:-1]), "%Y-%m-%d-%H-%M-%S-%f")
                timeChange = (timeNow - timePrevious).total_seconds()
                #print(timePrevious, timeNow)
                timePrevious = timeNow
                
                
                
                dwellTimeList.append(timeDwell)
                timeList.append(timeChange)
                
                #print('dec', key[0], timeDwell)
                
        else:
            userKeys[nKey].append(str(float(key[2])*timeDwell))
    
    time1, t1, t2 = userKeys[-1][0].partition('+')
    timeNow = datetime.datetime.strptime(re.sub('[:.T]','-',time1[:-1]), "%Y-%m-%d-%H-%M-%S-%f")
    
    timeChange = (timeNow - timePrevious).total_seconds()
    timeList.append(timeChange)
    
    #print('end', userKeys[-1][0])
    
    #print('')
    #print(dwellTimeList, timeList)
    
    #weighted_dwellTime = np.average(dwellTimeList, weights=timeList)
    #print(weighted_dwellTime)
    
    #print('')
    
    return dwellTimeList, timeList

In [67]:
class DataForEverySession:
    subjectID = ''
    blockNumber = ''
    sessionNumber = ''
    variable = ''
    dataForTrial = ''
    resultPathName = ''
   
    
    def printInfo(self):
        dataFrame = pd.DataFrame(list(zip([self.subjectID], [self.blockNumber], [self.sessionNumber], [self.dataForTrial])), columns=['subjectID', 'block', 'session', self.variable])
        
        return dataFrame
    
    def AddToFile(self):
        
        dataFrame = pd.DataFrame(list(zip([self.subjectID], [self.blockNumber], [self.sessionNumber], [self.dataForTrial])), columns=['subjectID', 'block', 'session', self.variable])
        
        book = load_workbook(self.resultPathName)
        writer = pd.ExcelWriter(self.resultPathName, engine='openpyxl')
        writer.book = book
        writer.sheets = dict((ws.title, ws) for ws in book.worksheets)
        startrow = writer.sheets['Sheet1'].max_row
        dataFrame.to_excel(writer, startrow = startrow, index = False, header = False)
        
        writer.save()

In [68]:
metricComputed_dwellTime = 'dwellTime'

resultFileName_dwellTime = r'C:\DTU\Data\201901_JanuaryExpt\DataExtracted\IndividualSessions_TrialAvg\Subject_Block_Session_' \
+ metricComputed_dwellTime +  '.xlsx'


dataFolderName = r'E:\Data\Data' # accessing external hard disk with the data
a = re.compile('(?<=Data\\\\Data\\\\)(.*)(?=\\\\[1-9])')
subjectName_listElement = 3

#dataFolderName = r'C:\DTU\Data\201901_JanuaryExpt' # accessing data saved in the computer
#a = re.compile('(?<=Data\\\\201901_JanuaryExpt\\\\)(.*)(?=\\\\[1-9])')
#subjectName_listElement = 4



for root, dirs, subfolder in os.walk(dataFolderName):
    
    technique = 'dwell_time'
    
    if not dirs:
        
        #if 'notCompleted' in root or 'notInclude' in root: # Some subjects do not have gaze log and have been marked as 
        
        if 'noData' in root or 'Trial' in root or 'trial' in root or 'Nothing' in root: # Some subjects do not have gaze log and have been marked as 
            #notInclude
            continue
        if 'Jonas' in root or 'Praktikant' in root or 'Villads' in root:
            continue
            
        if 'Picture' in root:
            continue
        #if '_MS' not in root:
        #    continue
        
        
        #if 'af\\' not in root:
        #    continue
        
        
        #if '2019-1-16-16-36-17_1stPart_2' not in root:
        #    continue
            
        keysSelected = None
        userKeys = None
        phraseLog = None
        
        for file in subfolder:
            
            
            if fnmatch.fnmatch(file, 'KeySelection*'):
                try:
                    
                    fKeysSelected = open(root + '\\' + file, encoding='utf-8', newline='')
                    readerKeysSelected = csv.reader(fKeysSelected)
                    keysSelected = list(readerKeysSelected)
                    
                    keysSelected.remove(keysSelected[0])
                except:
                    if fKeysSelected is not None:
                        fKeysSelected.close()
                    else:
                        print('error in opening the KeySelection log file')
                        
            
            if fnmatch.fnmatch(file, 'user*'):
                try:
                    fUserKey = open(root + '\\' + file, encoding='utf-8',  newline='')
                    readerUserKey = csv.reader(fUserKey, quotechar=None)
                    userKeys = list(readerUserKey)
                    userKeys.remove(userKeys[0])
                except:
                    if fUserKey is not None:
                        fUserKey.close()
                    else:
                        print('error in opening the user key log file')
                        
            if fnmatch.fnmatch(file, 'phrase*'):
                try:
                    fPhraseLog = open(root + '\\' + file, encoding='utf-8')
                    readerPhraseLog = csv.reader(fPhraseLog, quotechar=None)
                    phraseLog = list(readerPhraseLog)
                    
                except:
                    if fPhraseLog is not None:
                        fPhraseLog.close()
                    else:
                        print('error in opening the phrase log file')
                        
            if fnmatch.fnmatch(file, 'multiKey*'):
                technique = 'multiKey_selection'
            
            if fnmatch.fnmatch(file, 'tobiiGazeLog*'):
                try:
                    fGazeLog = open(root + '\\' + file, encoding='utf-8', newline='')
                    readerGazeLog = csv.reader(fGazeLog, quotechar=None)
                    gazeLog = list(readerGazeLog)
                    gazeLog.remove(gazeLog[0])
                    gazeLog.remove(gazeLog[-1])
                except:
                    if fGazeLog is not None:
                        fGazeLog.close()
                    else:
                        print('error in opening the scratchpad log file')
            
                    
                     
        if keysSelected is None or userKeys is None or phraseLog is None or gazeLog is None:
            continue
        else:
            
            print('subject path', root)
            subjAndSessionName = '__'.join(root.split('\\')[subjectName_listElement:])
            print(subjAndSessionName)
            subjName = subjAndSessionName.split('__')[0]
            print('subject and session name: ', subjAndSessionName)
            sessionFolderName = root.split('\\')[-1]
            
            # fix userKeys due to comma related file changes
            userKeys_new = FixUserKeys(userKeys)
            
            dwellTimeList, timeList = ComputeDwellTime(userKeys_new, root)
            
            if '1stPart' in root:
                dwellTimeList1 = dwellTimeList
                timeList1 = timeList
                continue
            
            if '2ndPart' in root:
                dwellTimeList2 = dwellTimeList
                timeList2 = timeList
                
                dwellTimeList = dwellTimeList1 + dwellTimeList2
                timeList = timeList1 + timeList2
                
                dwellTimeList1 = list()
                timeList1 = list()
                
            dwellTimeSession = np.average(dwellTimeList, weights=timeList)
            
            # save the writing start
            dataToSave_dwellTime = DataForEverySession()
            dataToSave_dwellTime.subjectID = subjAndSessionName.split('__')[0]
            dataToSave_dwellTime.blockNumber = subjAndSessionName.split('__')[1]
            dataToSave_dwellTime.sessionNumber = subjAndSessionName[-1]
            dataToSave_dwellTime.variable = metricComputed_dwellTime
            dataToSave_dwellTime.dataForTrial = dwellTimeSession
            dataToSave_dwellTime.resultPathName = resultFileName_dwellTime
            
            print(dataToSave_dwellTime.printInfo())
            dataToSave_dwellTime.AddToFile()

subject path E:\Data\Data\ac\1\2019-02-11-11-18-30_1
ac__1__2019-02-11-11-18-30_1
subject and session name:  ac__1__2019-02-11-11-18-30_1
  subjectID block session   dwellTime
0        ac     1       1  607.166677
subject path E:\Data\Data\ac\1\2019-02-11-11-33-32_2
ac__1__2019-02-11-11-33-32_2
subject and session name:  ac__1__2019-02-11-11-33-32_2
  subjectID block session   dwellTime
0        ac     1       2  501.544672
subject path E:\Data\Data\ac\2\2019-02-12-15-00-04_1
ac__2__2019-02-12-15-00-04_1
subject and session name:  ac__2__2019-02-12-15-00-04_1
  subjectID block session   dwellTime
0        ac     2       1  504.546946
subject path E:\Data\Data\ac\2\2019-02-12-15-27-37_2
ac__2__2019-02-12-15-27-37_2
subject and session name:  ac__2__2019-02-12-15-27-37_2
  subjectID block session   dwellTime
0        ac     2       2  419.270956
subject path E:\Data\Data\ac\3_MS\2019-02-14-14-28-49_1
ac__3_MS__2019-02-14-14-28-49_1
subject and session name:  ac__3_MS__2019-02-14-14-28-49

subject path E:\Data\Data\bh1\4\2019-01-31-09-12-2_1
bh1__4__2019-01-31-09-12-2_1
subject and session name:  bh1__4__2019-01-31-09-12-2_1
  subjectID block session   dwellTime
0       bh1     4       1  111.253536
subject path E:\Data\Data\bh1\4\2019-01-31-09-22-49_1stPart_2
bh1__4__2019-01-31-09-22-49_1stPart_2
subject and session name:  bh1__4__2019-01-31-09-22-49_1stPart_2
subject path E:\Data\Data\bh1\4\2019-01-31-09-37-5_2ndPart_2
bh1__4__2019-01-31-09-37-5_2ndPart_2
subject and session name:  bh1__4__2019-01-31-09-37-5_2ndPart_2
  subjectID block session   dwellTime
0       bh1     4       2  108.962567
subject path E:\Data\Data\bh1\5\2019-02-05-14-04-11_1
bh1__5__2019-02-05-14-04-11_1
subject and session name:  bh1__5__2019-02-05-14-04-11_1
  subjectID block session   dwellTime
0       bh1     5       1  105.751072
subject path E:\Data\Data\bh1\5\2019-02-05-14-20-6_2
bh1__5__2019-02-05-14-20-6_2
subject and session name:  bh1__5__2019-02-05-14-20-6_2
  subjectID block session   

  subjectID block session  dwellTime
0        jp     1       1      800.0
subject path E:\Data\Data\jp\1\2019-02-11-14-46-36_2
jp__1__2019-02-11-14-46-36_2
subject and session name:  jp__1__2019-02-11-14-46-36_2
  subjectID block session  dwellTime
0        jp     1       2      800.0
subject path E:\Data\Data\jp\2_MS\2019-02-12-10-18-24_1
jp__2_MS__2019-02-12-10-18-24_1
subject and session name:  jp__2_MS__2019-02-12-10-18-24_1
  subjectID block session  dwellTime
0        jp  2_MS       1      800.0
subject path E:\Data\Data\jp\2_MS\2019-02-12-10-46-29_2
jp__2_MS__2019-02-12-10-46-29_2
subject and session name:  jp__2_MS__2019-02-12-10-46-29_2
  subjectID block session  dwellTime
0        jp  2_MS       2      800.0
subject path E:\Data\Data\jp\3\2019-02-13-15-03-15_1
jp__3__2019-02-13-15-03-15_1
subject and session name:  jp__3__2019-02-13-15-03-15_1
  subjectID block session  dwellTime
0        jp     3       1      800.0
subject path E:\Data\Data\jp\3\2019-02-13-15-24-33_2
jp__3__

subject path E:\Data\Data\ls1\5\2019-02-21-17-11-42_1
ls1__5__2019-02-21-17-11-42_1
subject and session name:  ls1__5__2019-02-21-17-11-42_1
  subjectID block session   dwellTime
0       ls1     5       1  401.463414
subject path E:\Data\Data\ls1\5\2019-02-21-17-40-46_2
ls1__5__2019-02-21-17-40-46_2
subject and session name:  ls1__5__2019-02-21-17-40-46_2
  subjectID block session   dwellTime
0       ls1     5       2  304.566116
subject path E:\Data\Data\ls2\1\2019-02-11-14-24-40_1
ls2__1__2019-02-11-14-24-40_1
subject and session name:  ls2__1__2019-02-11-14-24-40_1
  subjectID block session  dwellTime
0       ls2     1       1      800.0
subject path E:\Data\Data\ls2\1\2019-02-11-14-53-55_2
ls2__1__2019-02-11-14-53-55_2
subject and session name:  ls2__1__2019-02-11-14-53-55_2
  subjectID block session  dwellTime
0       ls2     1       2      800.0
subject path E:\Data\Data\ls2\2\2019-02-12-10-02-45_1
ls2__2__2019-02-12-10-02-45_1
subject and session name:  ls2__2__2019-02-12-10-02-

subject path E:\Data\Data\no\3\2019-01-29-10-08-10_1
no__3__2019-01-29-10-08-10_1
subject and session name:  no__3__2019-01-29-10-08-10_1
  subjectID block session  dwellTime
0        no     3       1   538.0458
subject path E:\Data\Data\no\3\2019-01-29-10-23-27_2
no__3__2019-01-29-10-23-27_2
subject and session name:  no__3__2019-01-29-10-23-27_2
  subjectID block session   dwellTime
0        no     3       2  500.843847
subject path E:\Data\Data\no\4_MS\2019-01-30-10-20-32_1
no__4_MS__2019-01-30-10-20-32_1
subject and session name:  no__4_MS__2019-01-30-10-20-32_1
  subjectID block session  dwellTime
0        no  4_MS       1      800.0
subject path E:\Data\Data\no\4_MS\2019-01-30-10-46-38_2
no__4_MS__2019-01-30-10-46-38_2
subject and session name:  no__4_MS__2019-01-30-10-46-38_2
  subjectID block session  dwellTime
0        no  4_MS       2      800.0
subject path E:\Data\Data\no\5\2019-01-31-10-09-21_1
no__5__2019-01-31-10-09-21_1
subject and session name:  no__5__2019-01-31-10-09